In [ ]:
# imports
import os
os.environ['OMP_NUM_THREADS'] = '1'
import sys
sys.path.append('../')
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import trajectories
import noctiluca as nl
from tqdm import tqdm
from multiprocessing import Pool

# Generate some fBms

## fBm generator functions

Define some useful functions for generating the fBms.

In [ ]:
def generate_fgn(fbm_generator, sample_dimension, n_time, n_sample, file_path):
    """A helper function for generating fractional Gaussian noise samples.

    Parameters
    ----------
    fbm_generator : trajectories.FractionalBrownianMotion
        The FractionalBrownianMotion object for simulating the noise samples
    sample_dimension : int
        The spatial dimension for the samples (in this case 3)
    n_time : int
        The time dimension for the samples/number of time points desired.
    n_sample : int
        The number of trajectories to simulate.
    file_path : str
        The output path to save the .npy
    """
    fgn = fbm_generator.computation_method(fbm_generator.covariance_sequence,
                                           n_time,
                                           (sample_dimension, n_sample))
    np.save(file_path, fgn)

def generate_multiple_fgns(fbm_generator, sample_dimension, n_time, n_sample,
                           output_path, max_sample_size=1):
    """A helper function for generating multiple fgn samples.

    Parameters
    ----------
    fbm_generator : trajectories.FractionalBrownianMotion
        The FractionalBrownianMotion object for simulating the noise samples
    sample_dimension : int
        The spatial dimension for the samples (in this case 3)
    n_time : int
        The time dimension for the samples/number of time points desired.
    n_sample : int
        The number of trajectories to simulate.
    output_path : str
        The output path to save the .npy files
    max_sample_size : int, optional
        The maximum number of samples to generate at once, by default 1
    """
    sample_sizes = (n_sample // max_sample_size) * [max_sample_size] + \
        [n_sample % max_sample_size]
    file_prefix = f"fgn_hurst_{fbm_generator.hurst}_kr_{fbm_generator.k_r}_n_time_{n_time}"
    for i, sample_size in enumerate(sample_sizes):
        if sample_size > 0:
            generate_fgn(fbm_generator, sample_dimension, n_time, sample_size,
                         os.path.join(output_path, file_prefix + f"_{i}"))


## Generate fGns

In [ ]:
msd_exponents = [0.2, 0.3, 0.4, 0.5, 0.6] # exponents used for simulation

# a generator object for each exponent. Note that the covariance sequences for
# each generator will be cached to speed up computation.
fbm_generators = [trajectories.FractionalBrownianMotion(1, me / 2)
                   for me in msd_exponents]
sample_dimension = 3 # spatial dimension
n_time = 100000000 # number of time points
n_sample = 1000 # number of samples to generate
max_sample_size = 1 # maximum number of trajectories per simulation

output_path = "" # where to save the outputs
n_processes = 1 # number of threads to dedicate to multiprocessing

In [ ]:
for fbm_generator in tqdm(fbm_generators):
    sample_sizes = (n_sample // max_sample_size) * [max_sample_size] + \
        [n_sample % max_sample_size]
    file_prefix = f"fgn_hurst_{str(fbm_generator.hurst).replace(".", "p")}_kr_{
        str(fbm_generator.k_r).replace(".", "p")}_n_time_{n_time}"

    def parfun(i_sample_size):
        i, sample_size = i_sample_size
        if sample_size > 0:
            generate_fgn(fbm_generator, sample_dimension, n_time, sample_size,
                         os.path.join(output_path, file_prefix + f"_{i}"))

    with Pool(processes=n_processes) as mypool:
        mypool.map(parfun, list(enumerate(sample_sizes)))

## Examine some MSDs
Let's take a look at some of the MSDs to confirm they look like how we expect

In [ ]:
msd_fgn_paths = [""] # provide list of paths to files you would like to examine
msd_fbms = []
for msd_fgn_path in msd_fgn_paths:
    fgn = np.load(msd_fgn_path)
    fbm = np.cumsum(fgn, axis=0)
    fbm -= fbm[0, :, :]
    msd_fbms.append(fbm)

In [ ]:
colors = ['orangered', 'orchid', 'orange', 'gray', 'royalblue']
msd_exponents = [] # a list of the MSD exponent of each selected file
fig, ax = plt.subplots(figsize=(9, 6))
for color, exponent, fbm in zip(colors, msd_exponents, msd_fbms):
    data = nl.TaggedSet()
    fbm_split = np.split(fbm, 10000)
    for i in range(10):

        data.add(nl.Trajectory(np.squeeze(fbm_split[i])))

    artists = nl.plot.msd_overview(data, dt=1, label=f"{exponent}")

    for a in artists[:-1]:
        a.remove()
    artists[-1].set_color(color)

plt.xlabel("frame")
plt.ylabel("displacement")
plt.legend()
plt.show()

# Calculate first passage times

Here, we used a series of target distances from the origin combined with a 
series of target sizes to determine conditions for which the observed FPTs 
appeared to converge. 

## Define some useful functions

In [ ]:
def calculate_fpts(path, target_distances, target_sizes, output_path):
    """A helper function for calculating first passage times for a given
    trajectory.

    Parameters
    ----------
    path : str
        A path to a fgn sample as generated above
    target_distances : list | np.array
        An iterable of target distances from the origin
    target_sizes : list | np.array
        An iterable of target sizes
    output_path : str
        A path in which to save the intermediate fpt calculations

    Returns
    -------
    np.array
        The calculated fpts
    """
    path_split = path.split("/")[-1].split("_")
    hurst, traj_idx = path_split[2], path_split[-1].split(".")[0]

    fgn = np.load(path)
    fbm = np.cumsum(fgn, axis=0)
    fbm -= fbm[0, :, :]

    fpts = []

    for target_distance in target_distances:
        target_position = np.array([target_distance, 0, 0]).reshape((1, 3, 1))

        # calculate distances to target position for pts in fbm
        distances_to_target = np.linalg.norm(fbm - target_position, axis=1)
        for target_size in target_sizes:
            if target_size >= target_distance:
                fpt = 0
            else:
                fpt = np.sum(np.cumsum(distances_to_target < target_size, axis=0) == 0)
            fpts.append([target_distance, target_size, fpt,
                         float(hurst.split("p")[-1])/10, int(traj_idx)])

    out = np.array(fpts)
    np.save(os.path.join(output_path,
                         path.split("/")[-1]), out)
    return out

## Calculate FPTs

In [ ]:
fgn_lists = [] # provide a list of file paths to fgns

# define target distances and target sizes
target_distances = np.sqrt(3) * np.logspace(1, 3.5, num=15, base=2)
target_sizes = np.sqrt(3) * np.logspace(0, np.log2(5), num=8, base=2)

# output path
output_path = ""

# specify number of cores
n_processes = 1

def parfun(fgn_path):
    return calculate_fpts(fgn_path, target_distances, target_sizes,
                          output_path)

todo = fgn_lists
with Pool(processes=n_processes) as mypool:
    fpt_list = list(tqdm(mypool.imap(parfun, todo), total=len(todo)))

all_fpts = np.concatenate(fpt_list)


In [ ]:
fpt_df = []
for fpt_file_path in os.listdir(output_path):
    fpt_df.append(np.load(os.path.join(output_path, fpt_file_path)))

fpt_df = np.concatenate(fpt_df)
fpt_df = pd.DataFrame(fpt_df, columns=["target_distance", "target_size", "fpt", "alpha", "index"])

# correct some naminging
fpt_df.loc[fpt_df["alpha"] == 1.5, "alpha"] = 0.15
fpt_df.loc[fpt_df["alpha"] == 2.5, "alpha"] = 0.25

# convert H to alpha
fpt_df["alpha"] *= 2
fpt_df["status"] = True
fpt_df.loc[fpt_df["fpt"] == 100000000.0, "status"] = False
fpt_df.to_csv("fpts_from_simulation.csv")
fpt_df